### AutoKeras

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from imblearn.over_sampling import SMOTE
from tabulate import tabulate
import itertools
from copy import copy
#import sklearn.external.joblib as extjoblib
import joblib
#from sklearn.externals import joblib

import os
import tensorflow as tf
import autokeras as ak

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import ShuffleSplit
from sklearn.kernel_approximation import Nystroem
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.pipeline import make_pipeline, make_union
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
from sklearn.metrics import f1_score, precision_score, recall_score

### The LoadData routine is used to read the dataset, and create training and testing datasets

In [2]:
def LoadData():
    global feature_names, response_name, n_features

    model_full = pd.read_csv("C:/Users/affiqazrin/Desktop/mmlspark/Data_FinalProject_READY2.csv")

    response_name = ["indexedDeposit"]
    feature_names2 = ["job",
                    "marital",
                    "education",
                    "default",
                    "housing",
                    "loan",
                    "contact",
                    "day_of_week",
                    "month",
                    "poutcome",
                    "age"]
    
    feature_names = ["duration",
                     "campaign",
                     "pdays",
                     "previous"]
    n_features = len(feature_names)
    mask = feature_names + response_name

    model = model_full[mask]
    print('Model dataset:\n', model.head(5))
    # print('\nDescription of model dataset:\n', model[feature_names].describe(include='all'))

    # Correlation_plot(model)
    
    X = model[feature_names].values
    y = model[response_name].values.ravel()
    
    sm = SMOTE(random_state=12)
    X_resampled, y_resampled = sm.fit_sample(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled,
                                                        y_resampled,
                                                        test_size = 0.3,
                                                        random_state = 0)
    print('Size of resampled data:')
    print(' train shape... ', X_train.shape, y_train.shape)
    print(' test shape.... ', X_test.shape, y_test.shape)

    return X_train, y_train, X_test, y_test, X_resampled, y_resampled

### The ROC_Curve routine is used to plot the ROC curve.

In [3]:
def ROC_Curve():
    plt.figure()
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(false_positive_rate, true_positive_rate, color='darkorange', label='Random Forest')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve (area = %0.7f)' % auc)
    plt.legend(loc='best')
    plt.show()
    #path = "C:/Users/Charles/Desktop/ROC Curve.png"
    #plt.savefig(path)
    #plt.cla()
    #plt.close('all')

### This routine is used to plot the predictor (a.k.a feature) importances.

In [4]:
def Plot_predictor_importance(best_model, feature_names):
    feature_importance = best_model.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    y_pos = np.arange(sorted_idx.shape[0]) + .5
    fig, ax = plt.subplots()
    ax.barh(y_pos,
            feature_importance[sorted_idx],
            align='center',
            color='green',
            ecolor='black',
            height=0.5)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(feature_names)
    ax.invert_yaxis()
    ax.set_xlabel('Relative Importance')
    ax.set_title('Predictor Importance')
    plt.show()

### This routine is used to summarize the metrics for the model.

In [5]:
def Print_model_metrics(exported_pipeline, X_train, X_test, cm_train, cm_test, auc_train, auc_test):
    true_negative_train  = cm_train[0, 0]
    true_positive_train  = cm_train[1, 1]
    false_negative_train = cm_train[1, 0]
    false_positive_train = cm_train[0, 1]
    total_train = true_negative_train + true_positive_train + false_negative_train + false_positive_train
    accuracy_train = (true_positive_train + true_negative_train)/total_train
    precision_train = (true_positive_train)/(true_positive_train + false_positive_train)
    recall_train = (true_positive_train)/(true_positive_train + false_negative_train)
    misclassification_rate_train = (false_positive_train + false_negative_train)/total_train
    F1_train = (2*true_positive_train)/(2*true_positive_train + false_positive_train + false_negative_train)
    
    true_negative_test  = cm_test[0, 0]
    true_positive_test  = cm_test[1, 1]
    false_negative_test = cm_test[1, 0]
    false_positive_test = cm_test[0, 1]
    total_test = true_negative_test + true_positive_test + false_negative_test + false_positive_test
    accuracy_test = (true_positive_test + true_negative_test)/total_test
    precision_test = (true_positive_test)/(true_positive_test + false_positive_test)
    recall_test = (true_positive_test)/(true_positive_test + false_negative_test)
    misclassification_rate_test = (false_positive_test + false_negative_test)/total_test
    F1_test = (2*true_positive_test)/(2*true_positive_test + false_positive_test + false_negative_test)
    
    y_predict_train = exported_pipeline.predict(X_train)
    y_predict_test  = exported_pipeline.predict(X_test)
    mse_train       = metrics.mean_squared_error(y_train, y_predict_train)
    mse_test        = metrics.mean_squared_error(y_test, y_predict_test)
    logloss_train   = metrics.log_loss(y_train, y_predict_train)
    logloss_test    = metrics.log_loss(y_test, y_predict_test)
    accuracy_train  = metrics.accuracy_score(y_train, y_predict_train)
    accuracy_test   = metrics.accuracy_score(y_test, y_predict_test)
    precision_test  = precision_score(y_test, y_predict_test, average='binary')
    recall_test     = recall_score(y_test, y_predict_test, average='binary')
    F1_train        = metrics.f1_score(y_train, y_predict_train)
    F1_test         = metrics.f1_score(y_test, y_predict_test)
    r2_train        = metrics.r2_score(y_train, y_predict_train)
    r2_test         = metrics.r2_score(y_test, y_predict_test)
    auc_train       = metrics.roc_auc_score(y_train, y_predict_train)
    auc_test        = metrics.roc_auc_score(y_test, y_predict_test)
    
    header = ["Metric", "Train", "Test"]
    table = [["accuracy",               accuracy_train,               accuracy_test],
             ["precision",              precision_train,              precision_test],
             ["recall",                 recall_train,                 recall_test],
             ["misclassification rate", misclassification_rate_train, misclassification_rate_test],
             ["F1",                     F1_train,                     F1_test],
             ["r2",                     r2_train,                     r2_test],
             ["AUC",                    auc_train,                    auc_test],
             ["mse",                    mse_train,                    mse_test],
             ["logloss",                logloss_train,                logloss_test]
            ]

    print(tabulate(table, header, tablefmt="fancy_grid"))

### This routine plots the confusion matrix.

In [6]:
def Plot_confusion_matrix(cm, classes, title, normalize=False):
    cmap = plt.cm.Blues
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Load the training and test dataset

In [7]:
X_train, y_train, X_test, y_test, X, y = LoadData()

Model dataset:
    duration  campaign  pdays  previous  indexedDeposit
0       108         1    999         0             0.0
1       184         2    999         0             0.0
2        75         1    999         0             0.0
3       136         1    999         0             1.0
4       165         3    999         0             0.0
Size of resampled data:
 train shape...  (15395, 4) (15395,)
 test shape....  (6599, 4) (6599,)


In [8]:
def Run():
    aml = H2OAutoML(max_models = 500, 
                    seed = 7, 
                    max_runtime_secs = 1*60*60, 
                    nfolds = 5, 
                    stopping_metric = "misclassification",
                    project_name="bottle_rocket",
                    stopping_rounds = 5)
    
    aml.train(x = feature_columns, 
              y = "indexedDeposit",
              training_frame = train)

    lb = aml.leaderboard
    return lb, X_test, y_test, train, test, feature_columns, response_column

### Run the model obtained

In [9]:
clf = ak.StructuredDataClassifier(max_trials=5,overwrite=True,)

cbs = [tf.keras.callbacks.EarlyStopping(patience=3),]

clf.fit(
    x=X_train,
    y=y_train,
    epochs=200,
    callbacks=cbs,)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.7965823411941528

Best val_accuracy So Far: 0.7978968024253845
Total elapsed time: 00h 02m 37s
INFO:tensorflow:Oracle triggered exit
Epoch 1/200
482/482 [==============================] - 1s 1ms/step - loss: 0.5136 - accuracy: 0.7647
Epoch 2/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4568 - accuracy: 0.7903
Epoch 3/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4494 - accuracy: 0.7930
Epoch 4/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4441 - accuracy: 0.7932
Epoch 5/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4404 - accuracy: 0.7940
Epoch 6/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4378 - accuracy: 0.7940
Epoch 7/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4359 - accuracy: 0.7945
Epoch 8/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4343 - accuracy: 0.7950
Epoch 9/200


482/482 [==============================] - 1s 1ms/step - loss: 0.4185 - accuracy: 0.7978
Epoch 80/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4184 - accuracy: 0.7978
Epoch 81/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4184 - accuracy: 0.7979
Epoch 82/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4184 - accuracy: 0.7980
Epoch 83/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4184 - accuracy: 0.7980
Epoch 84/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4184 - accuracy: 0.7981
Epoch 85/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4184 - accuracy: 0.7981
Epoch 86/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4183 - accuracy: 0.7980
Epoch 87/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4183 - accuracy: 0.7978
Epoch 88/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4182 - accuracy: 

482/482 [==============================] - 1s 1ms/step - loss: 0.4175 - accuracy: 0.7979
Epoch 159/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4175 - accuracy: 0.7979
Epoch 160/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4175 - accuracy: 0.7979
Epoch 161/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4175 - accuracy: 0.7978
Epoch 162/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4175 - accuracy: 0.7979
Epoch 163/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4174 - accuracy: 0.7979
Epoch 164/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4174 - accuracy: 0.7979
Epoch 165/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4174 - accuracy: 0.7979
Epoch 166/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4174 - accuracy: 0.7980
Epoch 167/200
482/482 [==============================] - 1s 1ms/step - loss: 0.4173 - a

In [10]:
y_predicted_train = clf.predict(X_train)
y_predicted_test  = clf.predict(X_test)

# Since our target is (0,1), the classifier produces a probability matrix of (N,2).
# The first column refers to the probability that our data belong to class 0 (failure to reach altitude), 
# and the second column refers to the probability that the data belong to class 1 (it's a bottle rocket).
# Therefore, let's take the second column to compute the 'auc' metric.
#y_probabilities_test = clf.predict_proba(X_test)
#y_probabilities_success = y_probabilities_test[:, 1]

#false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_probabilities_success)
#auc = roc_auc_score(y_train, y_predicted_train)

y_predicted_train

array([[0.45255974],
       [0.24570936],
       [0.6517533 ],
       ...,
       [0.01208982],
       [0.05703884],
       [0.00971884]], dtype=float32)

In [12]:
# Evaluate the accuracy of the found model.
accuracy = clf.evaluate(X_test, y_test)[1]
print('Accuracy: {accuracy}%'.format(accuracy=round(accuracy * 100, 2)))

207/207 [==============================] - 0s 1ms/step - loss: 0.4379 - accuracy: 0.7900
Accuracy: 79.0%


In [14]:
#Get the best performing model & Summarize the loaded model
model = clf.export_model()
model.summary()

#Save the best performing model to file
model.save('duplicate_autoKeras_v2_SAVEDMODEL.tf')

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 4)                 0         
_________________________________________________________________
normalization (Normalization (None, 4)                 9         
_________________________________________________________________
dense (Dense)                (None, 32)                160       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
_________________________________________________________________
classification_head_1 (Activ (None, 1)                